# ZIP → LaTeX Concise Notes (via OpenRouter)

Этот ноутбук берёт ZIP-архивы **из текущей папки**, извлекает из каждого `main.tex`,
отправляет его содержимое в OpenRouter и получает **краткий конспект**,
который **сохраняет структуру и тэги**, но оставляет только пару базовых определений
и формул **без пояснений**. Результаты сохраняются в **отдельную папку**.

> ⚠️ **Безопасность ключа**: по умолчанию ноутбук берёт ключ из переменной окружения
`OPENROUTER_API_KEY`. Можно также вставить ключ вручную в параметрах ниже (не рекомендуется).


In [ ]:
# ==== Параметры ====
# Папка, куда будут сохраняться итоговые .tex с конспектами (создастся при отсутствии)
DEST_DIR = "summaries"

# Максимальное количество ZIP-архивов для обработки (чтобы не сжигать бюджет)
MAX_FILES = 100  # ← поменяйте по необходимости

# Название файла внутри ZIP (если лежит не в корне, скрипт найдёт по имени)
TARGET_TEX_BASENAME = "main.tex"

# Выберите модель OpenRouter (актуальные ID смотрите на https://openrouter.ai/models)
# Исправлено: корректный ID — с дефисом `qwen-2.5`, а не `qwen2.5`
MODEL_ID = "qwen/qwen-2.5-7b-instruct"
# Резервные модели, если основная недоступна/невалидна
MODEL_FALLBACKS = [
    "qwen/qwen-2.5-3b-instruct",
    "meta-llama/llama-3.1-8b-instruct",
    "mistralai/mistral-small-latest",
    "openai/gpt-4o-mini",
]

# Источник API-ключа. По умолчанию читаем из переменной окружения.
# Можно раскомментировать строку ниже и ВСТАВИТЬ ключ напрямую (менее безопасно).
import os

OPENROUTER_API_KEY = "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"  # ← ключ пользователя (не рекомендуется хранить в коде)
# OPENROUTER_API_KEY = os.environ.get("OPENROUTER_API_KEY", "")  # безопаснее брать из окружения

# Дополнительно: мягкий DRY-RUN. Если True — НИЧЕГО НЕ ОТПРАВЛЯЕТСЯ в сеть,
# а вместо запроса сохраняется заглушка (для проверки пайплайна).
DRY_RUN = False

# Ограничение на размер исходного LaTeX для отправки (символы).
# Если файл больше — будет обрезан по хвосту с уведомлением.
MAX_LATEX_CHARS = 120_000

# Управление печатью исходного main.tex в консоль
PRINT_LATEX_TO_CONSOLE = True
LATEX_PREVIEW_CHARS = 200000  # сколько символов показывать в консоли

In [30]:
# ==== Поиск ZIP-архивов в текущей папке ====
import glob
zip_paths = sorted([p for p in glob.glob("*.zip") if os.path.isfile(p)])
print(f"Найдено ZIP: {len(zip_paths)}")
for p in zip_paths[:10]:
    print(" •", p)
if len(zip_paths) > 10:
    print(" … и ещё", len(zip_paths)-10)


Найдено ZIP: 72
 • physics__7.avgspd.zip
 • physics__7.force.zip
 • physics__7arhimed.zip
 • physics__7friction.zip
 • physics__7graphs.zip
 • physics__7kinblok.zip
 • physics__7mgN.zip
 • physics__7pressure.zip
 • physics__7rpo.zip
 • physics__7springs.zip
 … и ещё 62


In [29]:
# ==== Подготовка промпта для модели ====
# Система и инструкция сформулированы так, чтобы модель
# возвращала краткий конспект с сохранением LaTeX-структуры.
SYSTEM_MSG = (
    "You are a LaTeX condenser. Given a LaTeX file (main.tex), "
    "produce a NEW LaTeX file that preserves the original structure, environments, and tags "
    "as much as reasonably possible, but replaces the content with a very short concise outline. "
    "The outline must contain only a few basic definitions and formulas, without explanations. "
    "Do not add commentary. Output ONLY the LaTeX content."
)

USER_TEMPLATE = (
    "Here is the original LaTeX file main.tex between <latex> tags. "
    "Create a concise version (keep structure and tags, keep it compilable). "
    "Only a couple of core definitions and formulas, no explanations.\n\n"
    "<latex>\n{latex}\n</latex>"
)


In [32]:
# ==== Обработка ZIP → main.tex → OpenRouter → summary.tex ====
import zipfile, requests, pathlib, csv, time, datetime, os

os.makedirs(DEST_DIR, exist_ok=True)
log_rows = []
ts = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

session = requests.Session()

# Обновленный системный промпт
SYSTEM_MSG = (
    "You are a strict LaTeX summarizer. "
    "Given a LaTeX file, output a NEW LaTeX document that preserves the same structure "
    "(documentclass, usepackage, sectioning, environments, math blocks). "
    "Keep exactly 1-2 of the most central formulas, and only a few short keywords or topic names "
    "to hint what the text is about. "
    "Do NOT include explanations, derivations, or comments. "
    "Do NOT remove LaTeX tags — your output must remain fully compilable."
)

def call_openrouter(api_key: str, model: str, system: str, user_content: str) -> str:
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "HTTP-Referer": "http://localhost",
        "X-Title": "Zip-Tex-Condenser-Notebook",
    }
    body = {
        "model": model,
        "messages": [
            {"role": "system", "content": system},
            {"role": "user", "content": user_content},
        ],
        "temperature": 0.0,  # минимизируем "творчество"
    }
    resp = session.post(url, headers=headers, json=body, timeout=120)
    resp.raise_for_status()
    return resp.json()["choices"][0]["message"]["content"]

def call_openrouter_with_fallbacks(api_key: str, models: list[str], system: str, user_content: str) -> str:
    last_err_txt = None
    for mid in models:
        try:
            print(f"  → Модель: {mid}")
            return call_openrouter(api_key, mid, system, user_content)
        except requests.HTTPError as e:
            err_text = getattr(e.response, 'text', '')[:200]
            print(f"    ↩︎ HTTP ошибка {mid}: {e} — {err_text}")
            last_err_txt = err_text
            time.sleep(0.5)
        except Exception as e:
            print(f"    ↩︎ Ошибка модели {mid}: {e}")
            last_err_txt = str(e)
            time.sleep(0.5)
    raise RuntimeError(f"Все модели не сработали. Последняя ошибка: {last_err_txt}")

def read_main_tex_from_zip(zip_path: str, basename: str = TARGET_TEX_BASENAME) -> str | None:
    with zipfile.ZipFile(zip_path, 'r') as zf:
        names = zf.namelist()
        candidates = [n for n in names if n.endswith("/"+basename) or n == basename]
        if not candidates:
            candidates = [n for n in names if n.split("/")[-1] == basename]
        if not candidates:
            return None
        with zf.open(candidates[0], 'r') as f:
            return f.read().decode("utf-8", errors="replace")

if not OPENROUTER_API_KEY:
    print("⚠️ OPENROUTER_API_KEY не найден. Установите переменную окружения или вставьте ключ в параметры.")
else:
    print("✅ Ключ найден (из параметров).")

# Модели: сначала GPT-4.1 (или gpt-4o-mini), потом Qwen
MODEL_PRIORITIES = [
    "openai/gpt-4.1-mini",
    "qwen/qwen2.5-7b-instruct",
]

processed = 0
for zip_path in zip_paths:
    if processed >= MAX_FILES:
        print(f"Достигнут лимит MAX_FILES={MAX_FILES}. Останавливаюсь.")
        break

    print("\n" + "—"*60)
    print(f"ZIP: {zip_path}")
    raw = read_main_tex_from_zip(zip_path)
    if raw is None:
        print("  ✖ main.tex не найден — пропуск.")
        log_rows.append([ts, zip_path, "", "main.tex not found", "0", "0"])
        continue

    if len(raw) > MAX_LATEX_CHARS:
        raw = raw[:MAX_LATEX_CHARS]

    user_msg = USER_TEMPLATE.format(latex=raw)

    try:
        if DRY_RUN:
            condensed = "% DRY RUN — сжатый LaTeX будет здесь\n\\documentclass{article}\n\\begin{document}\n...\n\\end{document}\n"
        else:
            condensed = call_openrouter_with_fallbacks(
                OPENROUTER_API_KEY,
                MODEL_PRIORITIES,
                SYSTEM_MSG,
                user_msg,
            )

        base = pathlib.Path(zip_path).stem
        out_name = f"{base}__summary.tex"
        out_path = os.path.join(DEST_DIR, out_name)

        with open(out_path, "w", encoding="utf-8") as f:
            f.write(condensed)

        print(f"  ✅ Сохранено: {out_path}")
        log_rows.append([ts, zip_path, out_path, "ok", str(len(raw)), str(len(condensed))])
        processed += 1

    except Exception as e:
        print(f"  ✖ Ошибка: {e}")
        log_rows.append([ts, zip_path, "", str(e), str(len(raw)), "0"])

# Запись лога
log_csv = os.path.join(DEST_DIR, "run_log.csv")
with open(log_csv, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["timestamp", "zip_path", "output_path", "status", "input_len", "output_len"])
    writer.writerows(log_rows)

print("\nИтог: обработано", processed, "архив(ов).")
print("Лог:", log_csv)


✅ Ключ найден (из параметров).

————————————————————————————————————————————————————————————
ZIP: physics__7.avgspd.zip
  → Модель: openai/gpt-4.1-mini
  ✅ Сохранено: summaries/physics__7.avgspd__summary.tex

————————————————————————————————————————————————————————————
ZIP: physics__7.force.zip
  → Модель: openai/gpt-4.1-mini
  ✅ Сохранено: summaries/physics__7.avgspd__summary.tex

————————————————————————————————————————————————————————————
ZIP: physics__7.force.zip
  → Модель: openai/gpt-4.1-mini
  ✅ Сохранено: summaries/physics__7.force__summary.tex
Достигнут лимит MAX_FILES=2. Останавливаюсь.

Итог: обработано 2 архив(ов).
Лог: summaries/run_log.csv
  ✅ Сохранено: summaries/physics__7.force__summary.tex
Достигнут лимит MAX_FILES=2. Останавливаюсь.

Итог: обработано 2 архив(ов).
Лог: summaries/run_log.csv


In [18]:
# ==== Просмотр результатов ====

import pandas as pd, os

log_csv = os.path.join(DEST_DIR, "run_log.csv")

# Список созданных файлов .tex
created = [p for p in os.listdir(DEST_DIR) if p.endswith(".tex")]
print("Создано .tex файлов:", len(created))
for p in created[:10]:
    print(" •", p)
if len(created) > 10:
    print(" … и ещё", len(created)-10)


Создано .tex файлов: 3
 • physics__7.avgspd__summary.tex
 • physics__7.force__summary.tex
 • physics__7arhimed__summary.tex
